<a href="https://colab.research.google.com/github/ilyesBoukraa/LU_SVD_decomposition/blob/main/LU_SVD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [2]:
import numpy as np
import scipy.linalg as la 
from scipy.linalg import hessenberg
import numpy.typing as npt
from collections.abc import Iterable
from typing import Union

# LU

In [3]:
# actually implemented PLU*
class LuDecomposition():
    def __init__(self)->None:
        pass

    def permutation_matrix(self, A:npt.NDArray[np.float64], identity_m:npt.NDArray[np.float64])->npt.NDArray[np.float64]:
            matrix_m = np.copy(A)
            n = len(A)

            #identity_m = np.diag(np.ones((n)))
            col_1 = matrix_m[0:n,0] 

            counter = 0
            for i in range(n-1):
                if( col_1[i] < col_1[i+1] and col_1[i]!=1 ):
                    #identity_M[i], identity_M[i+1]  = identity_M[i+1] , identity_M[i]
                    swap = np.copy(identity_m[0])
                    identity_m[0] = identity_m[i+1]
                    identity_m[i+1] = swap
                    counter+=1

            #print(identity_m)
            return identity_m #, counter


    # PLU
    def LU_decomposition(self,matrix:npt.NDArray[np.float64])->None:
        height,width = matrix.shape[:2]

        # not a squared matrix
        if(height != width ):
            print("False not Squared matrix")
            return None

        U = np.copy(matrix)
        U = U.astype('float64')

        L = np.identity(height , dtype = float)
        #L = np.diag(np.ones((height)))
        P = np.identity(height , dtype = float)
        
        # U upper
        # L lower
        #P = None
        for i in range(height):
            for j in range(i+1 , height):
                #swapping_nbr = 0
                #or (U[0:height,0][i] < U[0:height,0][i+1] )
                #U[i][i] == 0  or
                if( U[i][i] == 0  or (U[0:height,0][i] < U[0:height,0][i+1] )  ):
                    #P , swapping_nbr
                    P = self.permutation_matrix(U,P)
                    U = P @ U
                    
                number = U[j][i] / U[i][i]

                L[j][i] = number

                U[j] =  U[j] - number * U[i]
        
        print('########---P---#######')
        print(P)
        
        print('######---L---########')
        print(L)
    
        
        print('########---U---######')
        print(U)
        #return P,L, U 


# Testing The Results

In [4]:
np.random.seed(777)
A = np.random.randint(0, high=10, size=(3,3))

B = np.array([ [1,2,4],
               [6,5,7],
               [8,0,1] ])
A

array([[7, 6, 7],
       [1, 7, 4],
       [7, 9, 8]])

## Using my version.

In [5]:
TP = LuDecomposition()
TP.LU_decomposition(A)

########---P---#######
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
######---L---########
[[1.         0.         0.        ]
 [0.14285714 1.         0.        ]
 [1.         0.48837209 1.        ]]
########---L---######
[[ 7.          6.          7.        ]
 [ 0.          6.14285714  3.        ]
 [ 0.          0.         -0.46511628]]


## Using Scipy version.

In [6]:
p1,l1,u1 = la.lu(A)
print('----P----')
print(p1)
print('----L----')
print(l1)
print('----U----')
print(u1)

----P----
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
----L----
[[1.         0.         0.        ]
 [0.14285714 1.         0.        ]
 [1.         0.48837209 1.        ]]
----U----
[[ 7.          6.          7.        ]
 [ 0.          6.14285714  3.        ]
 [ 0.          0.         -0.46511628]]


# SVD

In [18]:
class singular_value_decomposition():
    def __init__(self)->None:
        self.epsilon = 10**-8
        self.precision = 10**-10
        
    def my_transpose(self ,matrix_or_vect:npt.NDArray[np.float64])->npt.NDArray[np.float64]:
        transposed_M = np.copy(matrix_or_vect)
        transposed_M_T = []
        if( len(transposed_M.shape) == 2 ):
            h,w = transposed_M.shape[:2] 
            
            for j in range(w):
                row = []
                for i in range(h):
                    row.append(transposed_M[i][j])
                transposed_M_T.append(row)

            transposed_M_T =np.array(transposed_M_T)
        else:
            h = transposed_M.shape[0]
            transposed_M_T = np.zeros((1,h))
            for i in range(h):
                transposed_M_T[0,i] = transposed_M[i] 

        return transposed_M_T

    def projection(self,x:npt.NDArray[np.float64], y:npt.NDArray[np.float64])-> float:
        
        denominator = self.my_transpose(y)@y
        if(denominator == 0):
            result = (self.my_transpose(x)@y) / ( denominator + self.epsilon )
        else:
            result = (self.my_transpose(x)@y) /  denominator
        
        return result


    def q_matrix(self, matrix:npt.NDArray[np.float64])->npt.NDArray[np.float64]:
        m = np.copy(matrix)
        height , width = m.shape[:2]
        Q = np.zeros(shape = (height,width) , dtype = float)  
        
        # q1 = v1
        Q[0:height,0] = m[0:height,0]

        for i in range(1,width):
            vector = m[0:height,i]
            q_i = vector
            for j in range(i-1,-1,-1):
                q_i =  q_i - self.projection( vector , Q[0:height,j] ) * Q[0:height,j] 

            Q[0:height,i] = q_i        

        for i in range(width):
            denominator = np.sqrt(( self.my_transpose(Q[0:height,i]) @ Q[0:height,i] ))
            Q[0:height,i] = Q[0:height,i] / (denominator + self.epsilon) 
        
        
        return Q

    def qr_decomposition(self,matrix:npt.NDArray[np.float64]):
        A = np.copy(matrix)
        A = A.astype('float')
        Q = self.q_matrix(A)
        R = self.my_transpose(Q) @ A
        return Q,R


    def eigen_values(self,my_matrix:npt.NDArray[np.float64], iterations=1_000, wilkinson=True, rayleigh=False)->npt.NDArray[np.float64]:
        #A = np.copy(my_matrix)    
        A = hessenberg(my_matrix)
        height , width = A.shape[:2]
        uk = 0
        #past = A[height-1,height-1]
        I =  np.identity(height, dtype = float)
        
        for i in range( int(iterations) ): 
            
            past = A[0,0]
            Q,R = self.qr_decomposition(A - uk * I)
            A_hat = R@Q + uk * I
            
            ##############################
            ######-wilkinson shift-#######
            #############################
            if(wilkinson):
                B = np.copy(A[height-2:height,height-2:height])
                eta = (B[0,0] - B[-1,-1]) /2
               
                eta_sign = 1
                if(eta<0):
                    eta_sign = -1
                denominator = ( np.abs(eta) + np.sqrt(eta**2 + B[0,1]**2) ) 
                if(denominator!=0):
                    uk = B[-1,-1] -  (eta_sign * B[0,1]**2)  / denominator 
                else:
                    uk = B[-1,-1] -  (eta_sign * B[0,1]**2)  / (denominator +self.epsilon)
            ##############################
            ###-end of wilkinson shift-###
            #############################
            
            
            ##############################
            ######-rayleigh shift-#######
            #############################
            if(rayleigh):
                uk = Q[0:height,height-1].T @ A @Q[0:height,height-1] 
                
            ##############################
            ######-end of rayleigh shift-#######
            ############################# 
            
            A = A_hat
            
            now = A[0,0]
            
            error = np.abs( past - now )
            #print(error)
            #self.epsilon
            if(error <= 10**-6 ):
                print('stopped in the iteration: ',i)
                break 
            
        eigen_vals = []
        for i in range(height):
            value = A[i][i]
            eigen_vals.append(value)
        
        eigen_vals = np.array(eigen_vals) 
        #eigen_vals = np.sort(eigen_vals)
        eigen_vals = self.sorting(eigen_vals)
        
        return eigen_vals
    
    def sorting(self, my_array:npt.NDArray[np.float64])->npt.NDArray[np.float64]:
        n = len(my_array)
        for i in range(n):
            for j in range(i+1,n):
                if (my_array[i]<my_array[j]):
                    my_array[i],my_array[j]=my_array[j],my_array[i]
        
        return my_array

    def backward_substitution(self,Ri:npt.NDArray[np.float64]) -> npt.NDArray[np.float64]:
        # Ax = b
        # A.T Ax = A.Tb
        # Rx = Q.T  b
        # since b = 0
        # Rx = 0
        # R is upper triangular
        # Find x
        matrix = np.copy(Ri)

        if(matrix[-1][-1] != 0):
            matrix[-1] = matrix[-1] / matrix[-1][-1] 
        if(matrix[-1][-1] == 0):
            matrix[-1][-1] = self.epsilon
            matrix[-1] = matrix[-1] / matrix[-1][-1] 
            
        n = len(matrix)
        x = np.zeros(n)
            
        x[n-1] = matrix[n-1,n-1]          
        for i in range(n-2,-1,-1):
            if(matrix[i][i] != 0):
                matrix[i,i:n] = matrix[i,i:n] /matrix[i][i]
            
            row = matrix[i,i:n]
            x[i] = (-row @ x[i:n]) / (matrix[i][i] + self.epsilon) 
        
        
        return x    
    
    def normalized_vect(self,  vect:npt.NDArray[np.float64] )->npt.NDArray[np.float64]:        
        denominator =  np.sqrt( np.sum(vect**2) )
        if(denominator == 0):
            normalized_v = vect / (denominator + self.epsilon)
        else:
            normalized_v = vect / (denominator )
        
        return normalized_v
     
    
    def eigen_vectors(self, matrix:npt.NDArray[np.float64])->npt.NDArray[np.float64]:
        A = np.copy(matrix)
        height , width = A.shape[:2]
        I = np.identity(height , dtype = float)
        
        eig_vals = self.eigen_values(A)
        #eig_vals = np.linalg.eigvals(A)
        vectors = []
        b = np.zeros(height)
        for i in range(len(eig_vals)):
            lamda = eig_vals[i] 
            if(lamda<=self.epsilon):
                lamda = 0
                
            new_matrix = A - lamda * I
            #-- mystrious line --#
        
            _, Ri = self.qr_decomposition(new_matrix)
            
            #_, Ri = np.linalg.qr(new_matrix)
            vect = self.backward_substitution(Ri)
                
            norm_vect = self.normalized_vect(vect)
            vectors.append(norm_vect)
            
        
        return self.my_transpose(np.array(vectors))
                                 
    def svd_decomposition(self, matrix:npt.NDArray[np.float64])->None:
        C = np.copy(matrix) 
        C = C.astype('float64')
        height, width = C.shape[:2]
        
        CTC = self.my_transpose(C) @ C
        CCT = C @ self.my_transpose(C)
        
        eig_vals = self.eigen_values( CTC)
        # putting the eigen values in the diagonal
        sigma = np.eye(height,width) * np.sqrt( np.abs(eig_vals) )
        
        v_t = self.eigen_vectors( CTC )
        v_t = self.my_transpose(v_t)
        
        u = self.eigen_vectors(CCT )
        
        print('########---U---##########')
        print(u)
        
        print('######---Sigma---########')
        print(sigma)
    
        
        print('########---V.T---########')
        print(v_t)
    

# Testing The Results

In [19]:
np.random.seed(777)
A = np.random.randint(0, high=10, size=(2,3))

B = np.array([ [1,2,4],
               [6,5,7],
               [8,0,1] ])

In [20]:
SVD = singular_value_decomposition()

## Using my version.

In [21]:
%%time
SVD.svd_decomposition(A)

stopped in the iteration:  4
stopped in the iteration:  4
stopped in the iteration:  2
########---U---##########
[[ 0.83783431 -0.54592459]
 [ 0.5459246   0.83783432]]
######---Sigma---########
[[13.57101477  0.          0.        ]
 [ 0.          3.97838634  0.        ]]
########---V.T---########
[[ 0.47238654  0.652013    0.5930683 ]
 [ 0.74996181 -0.65083992  0.11817226]
 [-0.46304237 -0.38895558  0.79643287]]
CPU times: user 10 ms, sys: 0 ns, total: 10 ms
Wall time: 12.5 ms


## Using numpy version.

In [16]:
%%time
s,v,d = np.linalg.svd(A)
print('---Numpy---')
print('-----U-----')
print(s)
print('----Sigma-----')
print(v)
print('----V.T-----')
print(d)

---Numpy---
-----U-----
[[-0.83783432 -0.54592459]
 [-0.54592459  0.83783432]]
----Sigma-----
[13.57101478  3.97838635]
----V.T-----
[[-0.47238655 -0.652013   -0.59306829]
 [-0.74996181  0.65083992 -0.11817226]
 [-0.46304237 -0.38895559  0.79643287]]
CPU times: user 3.86 ms, sys: 0 ns, total: 3.86 ms
Wall time: 22 ms


# Conclusions:

1.   this algorithm perform very well, could not beat numpy in terms of accuracy neither in speed. 
2.   definetly needs some changes, i don't think that the shifts supposed to be implemented this way, also since U and V are computed seperatly this algorithm won't give always the best results..
